# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686598


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:39,  3.55s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:09,  2.57s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:47,  1.84s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:33,  1.33s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:09<00:00,  8.68it/s]

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  9.01it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  7.91it/s]

Rendering models: 100%|██████████| 30/30 [00:10<00:00,  6.29it/s]

Anfrily                                0.002567
CThomas                                0.002339
not-logged-in-cbabef9bb5bb1fbb5b75     0.000452
Dynamatt                               0.000232
Daniel1a                               0.000316
mkehrli                                0.000170
kcarte13                               0.000196
justsomenoodles                        0.000157
jmfranci                               0.000368
raisamaduro                            0.019319
clairedeu                              0.000216
withadnotat                            0.000178
Toree_Carter                           0.000352
pangeli5                               0.000619
vctjones                               0.132192
not-logged-in-3ae935e3afaee843f530     0.038088
Lavadude                               0.015114
clars915                               0.000219
ShirleyL                               0.000228
27stefin                               0.010602
not-logged-in-9f43813670b3af3a0ed7     0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())